In [38]:
import cv2
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.utils import normalize
from tensorflow import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Activation,Dropout,Flatten,Dense

In [39]:
images_directory="datasets/"
no_tumor_images=os.listdir(images_directory+'no/')
dataset=[]
label=[]

INPUT_SIZE=64
# print(no_tumor_images)

yes_tumor_images=os.listdir(images_directory+'yes/')

# print(yes_tumor_images)

In [40]:
#chargement des images jpg chaque image 
        #va etre lu en format tableau numpy ensuit 
        # va convertir une image
        # enfin va redimentionner et charge dans DSet
for i,image_name in enumerate(no_tumor_images):
    if(image_name.split('.')[1]=='jpg'):
        image=cv2.imread(images_directory+'no/'+image_name)
        image=Image.fromarray(image,'RGB') #pour utiliser pillow on peux l'evite 
        image=image.resize((INPUT_SIZE,INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(0)

In [41]:
for i,image_name in enumerate(yes_tumor_images):
    if(image_name.split('.')[1]=='jpg'):
        image=cv2.imread(images_directory+'yes/'+image_name)
        image=Image.fromarray(image,'RGB')
        image=image.resize((INPUT_SIZE,INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(1)

In [21]:
print(len(dataset))
print(len(label))

3000
3000


In [42]:
dataset=np.array(dataset)
label=np.array(label)

x_train,x_test,y_train,y_test=train_test_split(dataset,label,test_size=0.2,random_state=0)

print(x_train.shape)

(2400, 64, 64, 3)


In [43]:
x_train=normalize(x_train,axis=1)
x_test=normalize(x_test,axis=1)

In [44]:
#Model Building

model=Sequential()

model.add(Conv2D(32,(3,3),input_shape=(INPUT_SIZE,INPUT_SIZE,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3),kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3),kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid')) #essayer avec softmax

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(x_train,y_train,
          batch_size=16,verbose=1,epochs=10,
         validation_data=(x_test,y_test),
         shuffle=False)
model.save('BrainTumorDetection.h5')

Epoch 1/10
150/150 [==============================] - 16s 82ms/step - loss: 0.5566 - accuracy: 0.7188 - val_loss: 0.4557 - val_accuracy: 0.8000
Epoch 2/10
150/150 [==============================] - 13s 84ms/step - loss: 0.4137 - accuracy: 0.8304 - val_loss: 0.4063 - val_accuracy: 0.8167
Epoch 3/10
150/150 [==============================] - 13s 86ms/step - loss: 0.3325 - accuracy: 0.8596 - val_loss: 0.3043 - val_accuracy: 0.8717
Epoch 4/10
150/150 [==============================] - 12s 83ms/step - loss: 0.2574 - accuracy: 0.8975 - val_loss: 0.2559 - val_accuracy: 0.9017
Epoch 5/10
150/150 [==============================] - 13s 87ms/step - loss: 0.1887 - accuracy: 0.9250 - val_loss: 0.2178 - val_accuracy: 0.9183
Epoch 6/10
150/150 [==============================] - 12s 83ms/step - loss: 0.1388 - accuracy: 0.9504 - val_loss: 0.1859 - val_accuracy: 0.9400
Epoch 7/10
150/150 [==============================] - 12s 82ms/step - loss: 0.0982 - accuracy: 0.9688 - val_loss: 0.1455 - val_accuracy:

In [57]:
#tester avec une image sur le web voir le fichier de teste elementaire 
imageP=cv2.imread('no_tumor.jpg')
imageP=Image.fromarray(imageP,'RGB') 
# Convertir l'image PIL en tableau numpy
image_array = np.array(imageP)

# Redimensionner l'image si nécessaire
image_array = cv2.resize(image_array, (INPUT_SIZE, INPUT_SIZE))

# Ajouter une dimension supplémentaire pour représenter le lot (batch) si nécessaire
image_array = np.expand_dims(image_array, axis=0)

# Faire la prédiction
prediction = model.predict(image_array)
print(prediction)

1/1 [==============================] - 0s 20ms/step
[[0.]]
